#### CSCE 489 :: Data Science and Analytics :: Texas A&M University :: Fall 2016

# Group Project: Analyzing Stack Over Flow Question 
### Team: StackPack
### Members: Stephen Brownlee, Eric Chung, Kyle Preston, Jacob Stafford



## Overview
- Clean and extract the data
- Analyze data
- Observe patterns and extract information
- Draw conclusions 

Data was found at:
    https://www.kaggle.com/stackoverflow/pythonquestions

Useful website: https://www.kaggle.com/jiashenliu/d/stackoverflow/pythonquestions/if-we-treat-stack-overflow-as-a-social-network

## Cleaning the data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np

answers = pd.read_csv('./answers.csv', encoding = 'iso-8859-1')
questions = pd.read_csv('./questions.csv', encoding = 'iso-8859-1')

answers['Length'] = answers.Body.str.len()
questions['Length'] = questions.Body.str.len()

answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,IsAcceptedAnswer,Body,Length
0,79741,3259.0,2008-09-17T03:43:22Z,79709,-1,False,<p>It's tough to say definitively without know...,328
1,79768,6043.0,2008-09-17T03:48:29Z,79709,5,False,<p>use variables in the outer function instead...,430
2,79779,8002.0,2008-09-17T03:49:36Z,79709,0,False,<p>Third approach: inner function returns a re...,341
3,79788,NaN,2008-09-17T03:51:30Z,79709,3,False,<p>It's not going to make much difference to m...,739
4,79827,14257.0,2008-09-17T03:58:26Z,79709,1,False,"<p>I'm not sure I understand the question, but...",1054


## Looking at word count 

In [3]:
#Filtering the answers, in which all of the tags are removed
answers['Filtered'] = answers.Body.str.strip().str.lower().str.replace('<p>', ' ').str.replace('</p>', ' ').str.replace('<code>', ' ').str.replace('</code>', ' ').str.replace('<blockquote>', ' ').str.replace('</blockquote>', ' ').str.replace('<pre>', ' ').str.replace('</pre>', ' ').str.replace('\n', ' ')
#Getting the length of the new filtered answers
answers['Length_Filtered'] = answers.Filtered.str.len()
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,IsAcceptedAnswer,Body,Length,Filtered,Length_Filtered
0,79741,3259.0,2008-09-17T03:43:22Z,79709,-1,False,<p>It's tough to say definitively without know...,328,it's tough to say definitively without knowin...,322
1,79768,6043.0,2008-09-17T03:48:29Z,79709,5,False,<p>use variables in the outer function instead...,430,use variables in the outer function instead o...,419
2,79779,8002.0,2008-09-17T03:49:36Z,79709,0,False,<p>Third approach: inner function returns a re...,341,third approach: inner function returns a refe...,330
3,79788,NaN,2008-09-17T03:51:30Z,79709,3,False,<p>It's not going to make much difference to m...,739,it's not going to make much difference to mem...,718
4,79827,14257.0,2008-09-17T03:58:26Z,79709,1,False,"<p>I'm not sure I understand the question, but...",1054,"i'm not sure i understand the question, but i...",1008


In [4]:
#Filtering the questions, in which all of the tags are removed
questions['Filtered'] = questions.Body.str.strip().str.lower().str.replace('<p>', ' ').str.replace('</p>', ' ').str.replace('<code>', ' ').str.replace('</code>', ' ').str.replace('<blockquote>', ' ').str.replace('</blockquote>', ' ').str.replace('<pre>', ' ').str.replace('</pre>', ' ').str.replace('\n', ' ')
#Getting the length of the new filtered questions
questions['Length_Filtered'] = questions.Filtered.str.len()
print len(questions.Body)
questions.head()

147075


,Id,OwnerUserId,CreationDate,Score,Title,Body,Length,Filtered,Length_Filtered
0,77434,14008.0,2008-09-16T21:40:29Z,134,How to access the last value in a vector?,<p>Suppose I have a vector that is nested in a...,405,suppose i have a vector that is nested in a d...,327
1,79709,NaN,2008-09-17T03:39:16Z,1,Worse sin: side effects or passing massive obj...,<p>I have a function inside a loop inside a fu...,1102,i have a function inside a loop inside a func...,1066
2,95007,15842.0,2008-09-18T17:59:19Z,48,Explain the quantile() function in R,<p>I've been mystified by the R quantile funct...,1919,i've been mystified by the r quantile functio...,1746
3,103312,NaN,2008-09-19T16:09:26Z,4,How to test for the EOF flag in R?,<p>How can I test for the <code>EOF</code> fla...,183,how can i test for the eof flag in r? f...,141
4,255697,1941213.0,2008-11-01T15:48:30Z,3,Is there an R package for learning a Dirichlet...,<p>I'm looking for a an <code>R</code> package...,485,i'm looking for a an r package which can be...,457


In [5]:
split_q = np.split(questions, 3)
print len(split_q[1])
print len(split_q[0])
print len(split_q[2])

test_q = split_q[0]
train_q = split_q[1]

49025
49025
49025


In [6]:
split_a = np.split(answers, 2)
print len(split_a[1])
print len(split_a[0])

train_a = split_a[0]
test_a = split_a[1]

99296
99296


## Using different classifiers to test how likely a question will be accepted.

In [7]:
x_train = train_a[['Length_Filtered']]
y_train = train_a[['IsAcceptedAnswer']]

x_test = test_a[['Length_Filtered']]
y_test = test_a[['IsAcceptedAnswer']]

In [8]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)
#Outputting training and testing data for word count of answers
print 'Training accuracy: ',clf.score(x_train,y_train)
print 'Testing accuracy: ',clf.score(x_test, y_test)

Training accuracy:  0.554272075411
Testing accuracy:  0.562902835965


/Users/Eric/anaconda2/envs/cs489/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)
print 'Training accuracy: ',clf.score(x_train,y_train)
print 'Testing accuracy: ',clf.score(x_test, y_test)

Training accuracy:  0.595200209475
Testing accuracy:  0.558461569449


In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
print 'Training accuracy: ',clf.score(x_train,y_train)
print 'Testing accuracy: ',clf.score(x_test, y_test)

/Users/Eric/anaconda2/envs/cs489/lib/python2.7/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


Training accuracy:  0.592974540767
Testing accuracy:  0.557726393812


## Length of an answer's relationship compared to that answer's score.

In [11]:
x_train = train_a[['Length_Filtered']]
y_train = train_a[['Score']]
x_test = test_a[['Length_Filtered']]
y_test = test_a[['Score']]

In [12]:
clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)
print 'Training accuracy: ',clf.score(x_train,y_train)
print 'Testing accuracy: ',clf.score(x_test, y_test)

Training accuracy:  0.272669593941
Testing accuracy:  0.272135836287


In [13]:
clf = MultinomialNB()
clf.fit(x_train, y_train)
print 'Training accuracy: ',clf.score(x_train,y_train)
print 'Testing accuracy: ',clf.score(x_test, y_test)

/Users/Eric/anaconda2/envs/cs489/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training accuracy:  0.230039477925
Testing accuracy:  0.289306719304


## Relationship between words used and the score of an answer or its likelihood of being accepted?

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer()

#take the first k answers
k = 1000
answer_text = answers['Filtered'][:k]
#TODO: break these up into multiple kFolds since the data is too large to do in one go
tf_train = tf_vectorizer.fit_transform(answer_text)
tf_test = tf_vectorizer.transform(answer_text)

target = answers['IsAcceptedAnswer'][:k]

clf = MultinomialNB()
clf.fit(tf_train, target)
print 'Answer Text -> IsAcceptedAnswer MultinomialNB Score: ',clf.score(tf_test, target)

clf = RandomForestClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> IsAcceptedAnswer RandomForest score: ',clf.score(tf_test, target)

clf = tree.DecisionTreeClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> IsAcceptedAnswer DecisionTree score: ',clf.score(tf_test, target)

target = answers['Score'][:k]

clf = MultinomialNB()
clf.fit(tf_train, target)
print 'Answer Text -> Score MutinomialBB score: ',clf.score(tf_test, target)

clf = RandomForestClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> Score RandomForest score: ',clf.score(tf_test, target)

clf = tree.DecisionTreeClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> Score DecisionTree score: ',clf.score(tf_test, target)

Answer Text -> IsAcceptedAnswer MultinomialNB Score:  0.884
Answer Text -> IsAcceptedAnswer RandomForest score:  0.969
Answer Text -> IsAcceptedAnswer DecisionTree score:  1.0
Answer Text -> Score MutinomialBB score:  0.565
Answer Text -> Score RandomForest score:  0.996
Answer Text -> Score DecisionTree score:  1.0


These numbers are surprisingly high! Is our model really that good, or is something wrong? Let's try using values that we suspect **should not be related to IsAcceptedAnswer** and see what score we get:

In [ ]:
target = answers['Id'][:k]

clf = MultinomialNB()
clf.fit(tf_train, target)
print 'Answer Text -> Id MultinomialBB score: ',clf.score(tf_test, target)

clf = RandomForestClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> Id RandomForest score: ',clf.score(tf_test, target)

clf = tree.DecisionTreeClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> Id DecisionTree score: ',clf.score(tf_test, target)

target = answers['Length_Filtered'][:k]

clf = MultinomialNB()
clf.fit(tf_train, target)
print 'Answer Text -> LengthF MultinomialBB score: ',clf.score(tf_test, target)

clf = RandomForestClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> LengthF RandomForest score: ',clf.score(tf_test, target)

clf = tree.DecisionTreeClassifier()
clf.fit(tf_train, target)
print 'Answer Text -> LengthF DecisionTree score: ',clf.score(tf_test, target)

### Why would there be a 0.999 score between our bag of words and User ID and or Length?..
Is something in RandomForest and DecisionTrees giving us false positives? Let's see how other classifiers fare when used with Score and IsAcceptedAnswer features.

In [19]:
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestCentroid

clf = MultinomialNB()
clf2 = SVC()
clf3 = LogisticRegression()
clf4 = NearestCentroid()
clf5 = SVR()

print ("Scores for words -> 'Score' ")


def fit_and_score(clf, train, test, target):
    clf.fit(train, target)
    print clf.score(test, target), "\n"

target = answers['Score'][:k]

print("MultinomialNB")
fit_and_score(clf, tf_train, tf_test, target)

print("Support vector classifier")
fit_and_score(clf2, tf_train, tf_test, target)

print("Logistic regression")
fit_and_score(clf3, tf_train, tf_test, target)

print("Nearest centroid")
fit_and_score(clf4, tf_train, tf_test, target)

print("SVR")
fit_and_score(clf5, tf_train, tf_test, target)

Scores for words -> 'Score' 
MultinomialNB
0.565 

Support vector classifier
0.136 

Logistic regression
0.998 

Nearest centroid
0.289 

SVR
-0.0361438614245 



In [16]:
target = answers['IsAcceptedAnswer'][:k]

print ("Scores for words -> 'IsAcceptedAnswer' ")

print("MultinomialNB")
fit_and_score(clf, tf_train, tf_test, target)

print("Support vector classifier")
fit_and_score(clf2, tf_train, tf_test, target)

print("Logistic regression")
fit_and_score(clf3, tf_train, tf_test, target)

print("Nearest centroid")
fit_and_score(clf4, tf_train, tf_test, target)

print("SVR")
fit_and_score(clf5, tf_train, tf_test, target)

Scores for words -> 'IsAcceptedAnswer' 
MultinomialNB
0.884 

Support vector classifier
0.709 

Logistic regression
0.994 

Nearest centroid
0.633 

SVR
-0.0338116930185 



### Looks like at least some other classifiers also give near-perfect scores... 
for our word vector's correlation to other features, but at least multiple others don't. Let's assume the classifiers not giving values >.95 are giving us the correct picture.


### Scaling Score 
Now, let's try to improve these classifiers' scores. First, we try alternate ways of defining 'Score.'  What if we sort score values into ** 5 custom-defined buckets? **


In [18]:
score_scaled = []
for answer in answers['Score']:
    if (answer<0):
        score_scaled.append(0)
    elif (answer < 1):
        score_scaled.append(1)
    elif (answer < 5):
        score_scaled.append(2)
    elif (answer < 15):
        score_scaled.append(3)
    else:
        score_scaled.append(4)
        
target = score_scaled[:k]

print("MultinomialNB")
fit_and_score(clf, tf_train, tf_test, target)

print("Support vector classifier")
fit_and_score(clf2, tf_train, tf_test, target)

print("Logistic regression")
fit_and_score(clf3, tf_train, tf_test, target)

print("Nearest centroid")
fit_and_score(clf4, tf_train, tf_test, target)

print("SVR")
fit_and_score(clf5, tf_train, tf_test, target)

MultinomialNB
0.841 

Support vector classifier
0.43 

Logistic regression
0.998 

Nearest centroid
0.266 

SVR
0.0279552547551 



#### Above, we can see this binning technique was a successful strategy to improve our score...
for all classifiers except **Nearest Centroid,** which became slightly worse. **SVR** seems overall unusable. It should also be noted that binning the data reduces the noise caused by the over-representation of the *-1* and *0* scores, but it also arguably **reduces the robustness of our data model,** as this distribution of SO scores might not necessarily pertain to all data sets collected from SO.

## Grouping the data by user ID

In [42]:
question_ids = questions['OwnerUserId'].unique()
answer_ids = answers['OwnerUserId'].unique()

combined_id=set(question_ids).intersection(answer_ids)
print 'So we have '+str(len(combined_id))+' users that post both questions and answers on StackOverFlow'

So we have 11194 users that post both questions and answers on StackOverFlow


In [43]:
users=pd.DataFrame({'idUser':list(combined_id)})
users.head(20)

users['Quantity']=users['idUser'].apply(lambda x: len(questions[questions['OwnerUserId']==x]['Body'])+len(answers[answers['OwnerUserId']==x]['Body']))
users=users.sort(['Quantity'],ascending=0).reset_index(drop=True)
users.head(8)

/Users/Eric/anaconda2/envs/cs489/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,idUser,Quantity
0,1855677.0,4997
1,1270695.0,2643
2,2372064.0,2314
3,143305.0,2245
4,1838509.0,2201
5,1412059.0,1895
6,190277.0,1838
7,3063910.0,1665


In [44]:
Q_data=questions[['OwnerUserId','Body']]
A_data=answers[['OwnerUserId','Body']]
question=Q_data[Q_data['OwnerUserId'].isin(users)]
answer=A_data[A_data['OwnerUserId'].isin(users)]

In [52]:
# An example of how we can inspect a specific user by their ID.
# We show all the SO data associated with that user ID.

highest = answers.loc[answers['OwnerUserId'] == 1855677.0]
highest.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,IsAcceptedAnswer,Body,Length,Filtered,Length_Filtered
568,1430603,1855677.0,2009-09-16T02:20:17Z,1428750,2,False,<p>?break</p>\n\n<p>Only gets you out of loop....,131,?break only gets you out of loop. ?try ...,110
748,1534985,1855677.0,2009-10-08T00:30:53Z,1529559,1,False,<pre><code> b&lt;-(y!=0)+0\n\n&gt; b\n [1] 0 1...,78,b&lt;-(y!=0)+0 &gt; b [1] 0 1 1 1 0 0 1 1...,57
749,1535082,1855677.0,2009-10-08T01:09:15Z,1523126,5,False,"<p>""Preprocess"" in R:</p>\n\n<pre><code>lines ...",863,"""preprocess"" in r: lines &lt;- ""www, rrr,...",775
1851,2483544,1855677.0,2010-03-20T15:15:12Z,2464680,1,False,"<p>The ""chi-square test"" is usually generated ...",619,"the ""chi-square test"" is usually generated as...",568
1852,2483836,1855677.0,2010-03-20T16:39:14Z,2478272,15,True,<p>One other approach besides kruskal::agricol...,2196,one other approach besides kruskal::agricolae...,2095


### Here, we just attempt to conglomerate all the SO data for a user ID into one text field.

In [49]:
combined_questions=[]
combined_answers=[]
user_ids=list(users['idUser'])
for id in user_ids:
    Q_frame=questions[questions['OwnerUserId']==id].reset_index(drop=True)
    A_frame=answers[answers['OwnerUserId']==id].reset_index(drop=True)
    for i in range(len(Q_frame['OwnerUserId'])):
        if i==0:
            tmp=Q_frame['Filtered'][i]
        else:
            tmp=tmp+Q_frame['Filtered'][i]
    combined_questions.append(tmp)
    
    for j in range(len(A_frame['OwnerUserId'])):
        if j==0:
            tmp2=A_frame['Filtered'][j]
        else:
            tmp2=tmp2+A_frame['Filtered'][j]
    combined_answers.append(tmp2)


In [50]:
Question_C=[]
Answer_C=[]

for i in range(11194):
    tmp1="".join(combined_questions[i])
    tmp2="".join(combined_answers[i])
    Question_C.append(tmp1)
    Answer_C.append(tmp2)

In [51]:
Final_frame=pd.DataFrame({'User_id':user_ids,'Question':Question_C,'Answer':Answer_C})
Final_frame.head()
series_a = Final_frame.loc[Final_frame['User_id'] == 1855677.0].Answer
string_series = series_a.to_string().replace('   ','')
print string_series

0  ?break only gets you out of loop. ?try ...
